In [2]:
bucket = 'vlgsagemakersimplemodel'
prefix = 'sagemaker/xgboost_credit_risk'

# Define IAM role
import boto3
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer

role = get_execution_role()

In [3]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls

--2020-02-05 19:04:41--  https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5539328 (5.3M) [application/x-httpd-php]
Saving to: ‘default of credit card clients.xls.1’

default of credit c 100%[===================>]   5.28M  8.87MB/s    in 0.6s    

2020-02-05 19:04:42 (8.87 MB/s) - ‘default of credit card clients.xls.1’ saved [5539328/5539328]



In [4]:
dataset = pd.read_excel('default of credit card clients.xls', index_col=None)
pd.set_option('display.max_rows', 8)
pd.set_option('display.max_columns', 15)
dataset

,Unnamed: 0,X1,X2,X3,X4,X5,X6,...,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,...,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,...,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,...,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,...,1518,1500,1000,1000,1000,5000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29997,29997,150000,1,3,2,43,-1,...,1837,3526,8998,129,0,0,0
29998,29998,30000,1,2,2,37,4,...,0,0,22000,4200,2000,3100,1
29999,29999,80000,1,3,1,41,1,...,85900,3409,1178,1926,52964,1804,1
30000,30000,50000,1,2,1,46,0,...,2078,1800,1430,1000,1000,1000,1


In [5]:
dataset = dataset.drop(dataset.columns[0], axis=1)
dataset = pd.concat([dataset['Y'], dataset.drop(['Y'], axis=1)], axis=1)
dataset

,Y,X1,X2,X3,X4,X5,X6,...,X17,X18,X19,X20,X21,X22,X23
0,default payment next month,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,...,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
1,1,20000,2,2,1,24,2,...,0,0,689,0,0,0,0
2,1,120000,2,2,2,26,-1,...,3261,0,1000,1000,1000,0,2000
3,0,90000,2,2,2,34,0,...,15549,1518,1500,1000,1000,1000,5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29997,0,150000,1,3,2,43,-1,...,0,1837,3526,8998,129,0,0
29998,1,30000,1,2,2,37,4,...,19357,0,0,22000,4200,2000,3100
29999,1,80000,1,3,1,41,1,...,48944,85900,3409,1178,1926,52964,1804
30000,1,50000,1,2,1,46,0,...,15313,2078,1800,1430,1000,1000,1000


In [6]:
train_data, validation_data, test_data = np.split(dataset.sample(frac=1, random_state=1729), [int(0.7 * len(dataset)), int(0.9 * len(dataset))])

In [9]:
train_data.to_csv('train.csv', header=False, index=False)
test_data.to_csv('test.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)

In [10]:
s3_input_train = boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_validation = boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

In [11]:
region = boto3.Session().region_name

sess = sagemaker.Session()

In [12]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(region, 'xgboost', '0.90-1')

In [14]:
xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)

In [16]:
xgb.set_hyperparameters(eta=0.1,
                        objective='binary:logistic',
                        num_round=25)

In [17]:
s3_training_file_location  = 's3://vlgsagemakersimplemodel/sagemaker/xgboost_credit_risk/train'
s3_validation_file_location = 's3://vlgsagemakersimplemodel/sagemaker/xgboost_credit_risk/validation'
training_input_config = sagemaker.session.s3_input(s3_data=s3_training_file_location,content_type="csv")
validation_input_config = sagemaker.session.s3_input(s3_data=s3_validation_file_location,content_type="csv")

print(training_input_config)
print(validation_input_config)

In [18]:
xgb.fit({'train': training_input_config, 'validation': training_input_config})

2020-02-05 19:06:36 Starting - Starting the training job...
2020-02-05 19:06:38 Starting - Launching requested ML instances......
2020-02-05 19:07:42 Starting - Preparing the instances for training......
2020-02-05 19:09:07 Downloading - Downloading input data
2020-02-05 19:09:07 Training - Downloading the training image.....INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[19:09:40] 21000x23 matrix with 483000 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Determined delimiter o

In [19]:
xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

---------------------!

In [22]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.values[:, 1:])
predictions

array([0.10145303, 0.29316297, 0.70782304, ..., 0.15233986, 0.38610485,
       0.10773233])

In [23]:
xgb_predictor.delete_endpoint()